In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.manifold import TSNE
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

SELF_DEFINED_STOP_WORDS = ['thy', 'thou', 'thee', 'sir', 'shall']
TOPIC_COLOR_MAP = np.array([
    "#1f77b5", "#aec7e9", "#ff7f0f", "#ffbb79", "#2ca02d", "#98df8b", "#d62729", "#ff9897", "#9467be", "#c5b0d6",
    "#8c564c", "#c49c95", "#e377c3", "#f7b6d3", "#7f7f80", "#c7c7c8", "#bcbd23", "#dbdb8e", "#17bed0", "#9edae6"
])

In [5]:
df = pd.read_csv("../data/processed/processed.csv")
df.dropna(subset=['PlayerLine'], inplace=True)
print "Load the processed data of the Shakespear playerline data set"
df.head()

Load the processed data of the Shakespear playerline data set


,Dataline,Play,Play_length,PlayerLinenumber,ActSceneLine,ASL_1,ASL_2,ASL_3,Player,PL_length,PL_w_count,PL_w_density,PlayerLine,PL_contain_!,PL_contain_?,PL_#_comma_split,PL_#_stop_words,PL_#_upper_case
0,1,Henry IV,8,NaN,NaN,NaN,NaN,NaN,NaN,5,2,2.500000,ACT I,False,False,1,1,4
1,2,Henry IV,8,NaN,NaN,NaN,NaN,NaN,NaN,28,5,5.600000,SCENE I. London. The palace.,False,False,1,1,8
2,3,Henry IV,8,NaN,NaN,NaN,NaN,NaN,NaN,96,16,6.000000,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ...",False,False,5,4,59
3,4,Henry IV,8,1.0,1.1.1,1.0,1.0,1.0,KING HENRY IV,38,9,4.222222,"So shaken as we are, so wan with care,",False,False,3,5,1
4,5,Henry IV,8,1.0,1.1.2,1.0,1.0,2.0,KING HENRY IV,42,9,4.666667,"Find we a time for frighted peace to pant,",False,False,2,4,1


Training LDA model on the playerline text

In [8]:
playerlines = df['PlayerLine']
count_vectorizer_limited = CountVectorizer(stop_words='english', max_features=40000)
document_term_matrix = count_vectorizer_limited.fit_transform(playerlines)
print "term matrix generated"
LDA_model = LatentDirichletAllocation(learning_method='online', n_components=14, random_state=0, verbose=0)
print "LDA model initialized"
topic_matrix = LDA_model.fit_transform(document_term_matrix)
print "LDA model trained"

term matrix generated
LDA model initialized
LDA model trained


In [ ]:
Use tsne to reduce dimension to 2, and visualize 2-dimention data points

In [ ]:
tsne_model = TSNE(n_components=2, perplexity=50, learning_rate=100, n_iter=2000, verbose=0, random_state=0, angle=0.75)
tsne_vectors = tsne_model.fit_transform(topic_matrix)

def get_keys(topic_matrix):
    keys = topic_matrix.argmax(axis=1).tolist()
    return keys

def _get_mean_topic_vectors(keys, two_dim_vectors):
        '''
        returns a list of centroid vectors from each predicted topic category
        '''
        centroid_topic_vectors = []
        for t in range(self.n_components):
            articles_in_that_topic = []
            for i in range(len(keys)):
                if keys[i] == t:
                    articles_in_that_topic.append(two_dim_vectors[i])

            articles_in_that_topic = np.vstack(articles_in_that_topic)
            centroid_article_in_that_topic = np.mean(articles_in_that_topic, axis=0)
            centroid_topic_vectors.append(centroid_article_in_that_topic)
        return centroid_topic_vectors

n_components = 10
keys = get_keys(topic_matrix)
mean_topic_vectors = _get_mean_topic_vectors(keys, tsne_vectors)

In [ ]:
def get_top_n_words(n, n_topics, document_term_matrix, keys, count_vectorizer):
    top_word_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:], 0)
        top_word_indices.append(top_n_word_indices)
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1, document_term_matrix.shape[1]))
            temp_word_vector[:, index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))
    return top_words

colormap = TOPIC_COLOR_MAP[:n_components]
top_3_words = get_top_n_words(3, n_components, document_term_matrix, keys, count_vectorizer)
fig, ax = plt.subplots(figsize=(16, 16))
plt.scatter(x=tsne_vectors[:, 0], y=tsne_vectors[:, 1],color=colormap[self.keys],marker='o', s=sampled_up_votes_sizes, alpha=0.5)
for t in range(self.n_components):
    plt.text(mean_topic_vectors[t][0], mean_topic_vectors[t][1], top_3_words[t], color=colormap[t],  horizontalalignment='center', weight='bold')
plt.show()